In [3]:
import boto3
import pandas as pd
import plotly.express as px

s3 = boto3.client('s3')

In [4]:
## pull all the files from a specific bucket and file prefix 
def get_files(bucket, prefix):
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    files = []
    for obj in response.get('Contents', []):
        files.append(obj.get('Key'))
    return files

## take in the files, read them in, and concatenate them into a single dataframe
def read_files(bucket, files):
    dfs = []
    for file in files:
        obj = s3.get_object(Bucket=bucket, Key=file)
        df = pd.read_csv(obj['Body'])
        dfs.append(df)
    return pd.concat(dfs)

In [5]:
file_names = get_files('icarus-research-data', 'PTST_weekly_predictions/first_run')
df = read_files('icarus-research-data', file_names)

In [19]:
data = df.copy()
data = data.loc[data['symbol'] != 'META']
data['real_price_change'] = ((data['target'] - data['alert_price'])/data['alert_price']) * 100
data['predicted_price_change'] = ((data['prediction'] - data['alert_price'])/data['alert_price'])*100
data['residuals'] = (data['real_price_change'] - data['predicted_price_change'])/data['real_price_change']
data['year'] = data['date'].apply(lambda x: x.split('-')[0])

In [20]:
data.head()

,Unnamed: 0,symbol,date,hour,minute,upside_threshold_prediction,alert_price,strategy,downside_threshold_prediction,dt,target,prediction,target_index,prediction_index,real_price_change,predicted_price_change,residuals,year
0,0,DIS,2024-01-19,10,0,1,93.2234,CDGAINC_2H,0,2024-01-19,95.8900,92.989357,27,41,2.860441,-0.251056,1.087768,2024
1,1,GOOG,2024-01-18,11,0,1,144.8500,CDGAINC_2H,1,2024-01-18,150.0150,146.232773,25,39,3.565758,0.954624,0.732280,2024
3,3,QQQ,2024-01-18,12,0,1,412.4219,CDGAINC_2H,1,2024-01-18,424.7275,414.955662,23,39,2.983741,0.614362,0.794097,2024
4,4,AMD,2024-01-16,10,0,1,154.0900,CDGAINC_2H,1,2024-01-16,168.6000,155.386203,27,39,9.416575,0.841199,0.910668,2024
5,5,GOOGL,2024-01-19,11,30,1,146.0750,CDGAINC_2H,0,2024-01-19,149.8550,146.218330,41,39,2.587712,0.098121,0.962082,2024


In [21]:
data.columns

Index(['Unnamed: 0', 'symbol', 'date', 'hour', 'minute',
       'upside_threshold_prediction', 'alert_price', 'strategy',
       'downside_threshold_prediction', 'dt', 'target', 'prediction',
       'target_index', 'prediction_index', 'real_price_change',
       'predicted_price_change', 'residuals', 'year'],
      dtype='object')

In [14]:
fig = px.scatter(
    data, x='predicted_price_change', y='real_price_change',
    title='Real vs Predicted Price Change',hover_data=['symbol'],
                 )
fig.show()

In [17]:
data.groupby('symbol')['residuals'].describe()

,count,mean,std,min,25%,50%,75%,max
symbol,,,,,,,,
AAPL,1175.0,-inf,NaN,-inf,-2.334445,-0.403017,0.292751,4.100331
AMD,1178.0,-inf,NaN,-inf,-3.268892,-0.617337,0.347230,21.511595
AMZN,1145.0,-4.578214,30.001106,-7.988489e+02,-2.251974,-0.321416,0.367935,78.833056
BA,1215.0,-inf,NaN,-inf,-2.586744,-0.531778,0.289846,456.372030
BAC,21.0,-6.315042,16.017203,-6.095928e+01,-3.226325,-0.315479,0.975065,3.036171
C,15.0,-1.892572,7.022058,-2.720851e+01,-0.191808,0.015351,0.189671,0.455734
DIS,1181.0,NaN,NaN,-inf,-4.735040,-1.236700,0.077153,inf
GOOG,1195.0,-4.356352,34.783981,-9.175894e+02,-2.434299,-0.421013,0.351828,182.977704
GOOGL,1154.0,-3.686241,14.805441,-2.289692e+02,-2.548743,-0.530165,0.340990,76.440370


In [22]:
data.groupby('year')['residuals'].describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2021,5666.0,NaN,NaN,-inf,-1.995901,-0.237032,0.416341,inf
2022,4687.0,-inf,NaN,-inf,-3.108162,-0.743334,0.165449,1711.378049
2023,5763.0,-inf,NaN,-inf,-2.656430,-0.512149,0.296612,879.928423
2024,3203.0,-inf,NaN,-inf,-2.149782,-0.225971,0.428069,4244.069069
